apply the bilateral blur then sustract it from the original image then add the result to the original image

In [42]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

hdr_img = cv.imread("image/image.tif", -1)
cv.imshow('TiffExemple',hdr_img)
cv.waitKey(0)
cv.destroyAllWindows()


In [69]:

img8 = (hdr_img/256).astype('uint8')
#print(img8.dtype)
blur = cv.bilateralFilter(img8,9,75,75)  #cv.bilateralFilter(src, d, sigmaColor, sigmaSpace[, dst[, borderType]])
 
cv.imshow('TiffExemple',(img8-blur)+img8)
cv.waitKey(0)
cv.destroyAllWindows()

def ApplyBilateralFilter(img):
    blur = cv.bilateralFilter(img,9,75,75)
    result = img + (img-blur) 
    return result

ApplyBilateralFilter(img8)


array([[244, 244, 243, ..., 218, 218, 218],
       [ 89,  85,  83, ...,  59,  59,  59],
       [ 93,  89,  84, ...,  65,  65,  65],
       ...,
       [ 69,  69,  67, ...,  69,  63,  54],
       [ 69,  67,  67, ...,  66,  62,  57],
       [255, 255,   0, ..., 222, 222, 222]], dtype=uint8)

# Toolbox to get weight maps, slices and fusion

In [85]:
from Bilateral_histogram_equalisation import showImageHDR
from typing import List

def getSlices(bins: int) -> List[np.ndarray]:
    """Cut image into equal sized of bins

    Returns:
        List[nd.array]: image slices
    """
    x = np.linspace(0,65535,30)
    sk = []
    for i in range(bins-1):
        null_compare = hdr_img[:,:] >= x[i+1]

        if i > 0: 
            slice = (hdr_img - x[i]) / (x[i+1] - x[i])
            slice = slice*null_compare
            m, n = np.where(hdr_img >= x[i])
            slice[m,n] = hdr_img[m, n]
        else:
            slice = hdr_img
        sk.append(slice)
    return sk

def getWeightMap() -> List[np.ndarray]:
    """Find the best candidate for the weight map wk

    Returns:
        List[np.ndarray]: list of weight maps
    """
    wk = []
    for i in range(29):
        one_compare = hdr_img[:,:] <= x[i]
        null_compare = hdr_img[:,:] >= x[i+1]
        other_compare = (~ one_compare)*(~ null_compare)

        ik = other_compare*hdr_img
        # ik = (ik/256).astype('uint8')
        # ik = ApplyBilateralFilter(ik)
        wk.append(ik)
    return wk

def fusion(weightMap: List[np.ndarray], slices: List[np.ndarray]) -> np.ndarray: 
    output = []
    for i in range(29):
        output.append(weightMap[i] @ slices[i])
    result = output[0]
    for i in range(1, 29):
        result += output[i].astype('uint16')
    showImageHDR(result)
    return result

In [86]:
sk = getSlices(bins=30)
wk = getWeightMap()
fusion(wk, sk)

array([[ 5705,  9757, 32624, ..., 10852, 57894, 15528],
       [23480, 58681, 42344, ...,  2823,   861, 39286],
       [24009, 11522, 19280, ..., 12479, 64282, 22061],
       ...,
       [43424, 38875, 53596, ..., 17545, 44102, 48481],
       [12607, 53673, 36029, ..., 53801, 22816, 62069],
       [43573, 64809, 62195, ..., 32260, 24061, 13485]], dtype=uint16)

Equalize(stretch) each slice

In [24]:
slices_stretch = []
max_bit = 2**16
for slc in sk :

    hist,bins = np.histogram(slc.flatten(),max_bit,[0,max_bit])
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()

    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*max_bit/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint16')

    slices_stretch.append(cdf[slc])

print(np.array(slices_stretch).shape) # Show shape

columns = 5
rows = 6
fig = plt.figure(figsize=(20, 20))
for i in range(1, columns*rows +1):
    img = sk[i-2]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()


cv.imshow('TiffStretchExemple',sk[5])
cv.waitKey(0)
cv.destroyAllWindows()

cv.imshow('TiffStretchExemple',slices_stretch[5])
cv.waitKey(0)
cv.destroyAllWindows()


IndexError: arrays used as indices must be of integer (or boolean) type